## Example Usage of the GMail Python API

This Notebook shows the basic usage of the GMail API

1. Load E-Mails from the server

2. Send E-Mails with the server

In order to be able to use the API you need a valid token as a .pickle file. In the first initialization of the GMail service you need to pass the path to this token.pickle file. Change the parameter below to your file to rerun the examples. 


For furhter information on how to send attachments, ... please consult the Documentation: 

https://googleapis.github.io/google-api-python-client/docs/epy/index.html

and the Guide: 

https://developers.google.com/gmail/api/quickstart/python

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from googleapiclient.discovery import build, Resource
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import pickle
import os
from typing import Dict
import email
import pandas as pd

### Basic Functions

In [3]:
def load_service(token_path:str):
    """ Load the service from the specifies token file

    Args:
        token_path (str): Authorized Gmail API service instance.
    """
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
            service = build('gmail', 'v1', credentials=creds)
            return service

    else: 
        print("Credentials not found")
        return None

In [4]:
def send_message(service:Resource, user_id:str, message:Dict[str,str]):
    """Send an email message.

    Args:
        service (Resource): Authorized Gmail API service instance.
        user_id (str): User's email address. The special value "me" can be used to indicate the authenticated user.
        message (Dict[str,str]): Message to be sent.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                  .execute())
        print('Message Id: %s' % message['id'])
    except Exception as e:
        print('An error occurred: %s' % e)

In [5]:
def create_message(sender:str, to:str, subject:str, message_text:str) -> Dict[str,str]:
  """Create a message for an email.

  Args:
    sender (str): Email address of the sender.
    to (str): Email address of the receiver.
    subject (str): The subject of the email message.
    message_text (str): The text of the email message.

  Returns:
    Dict[str,str]: An object containing a base64url encoded email object.
  """
  message = MIMEMultipart() #MIMEText(message_text, "html", _charset="utf-8")
  #message.set_payload(message_text, charset="utf-8")
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  message.attach(MIMEText(message_text, _charset='utf-8'))

  print(message)

  encoded_message = base64.urlsafe_b64encode(message.as_bytes())
  print(encoded_message)
  print(encoded_message.decode())
  return {'raw': encoded_message.decode()}


In [6]:
message = create_message("datathon@lumos-consulting.at", "thomas.n71@gmail.com", "Test_0", "Das ist der erste Testä")

Content-Type: multipart/mixed; boundary="===============2922927868841235215=="
MIME-Version: 1.0
to: thomas.n71@gmail.com
from: datathon@lumos-consulting.at
subject: Test_0

--===============2922927868841235215==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

RGFzIGlzdCBkZXIgZXJzdGUgVGVzdMOk

--===============2922927868841235215==--

b'Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT0yOTIyOTI3ODY4ODQxMjM1MjE1PT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB0aG9tYXMubjcxQGdtYWlsLmNvbQpmcm9tOiBkYXRhdGhvbkBsdW1vcy1jb25zdWx0aW5nLmF0CnN1YmplY3Q6IFRlc3RfMAoKLS09PT09PT09PT09PT09PT0yOTIyOTI3ODY4ODQxMjM1MjE1PT0KQ29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1dGYtOCIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogYmFzZTY0CgpSR0Z6SUdsemRDQmtaWElnWlhKemRHVWdWR1Z6ZE1PawoKLS09PT09PT09PT09PT09PT0yOTIyOTI3ODY4ODQxMjM1MjE1PT0tLQo='
Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT0yOTIyOTI3ODY4ODQxMjM1MjE1

In [7]:
def receive_unread_messages(service:Resource):
    """ Recieve all unread messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        if "UNREAD" in message["labelIds"]:
            try:
                print(message["id"])
                msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
                print(msg_str)
                mime_msg = email.message_from_string(msg_str)
                
                messages.append((message["id"],mime_msg))
    
            except: 
                print("error")
                pass
    return messages

In [8]:
def receive_all_messages(service:Resource):
    """ Recieve all messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        
        try:
            print(message["id"])
            msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
            print(msg_str)
            mime_msg = email.message_from_string(msg_str)

            messages.append(mime_msg)

        except: 
            print("error")
            pass
    return messages

In [9]:
# print_message(message)

In [10]:

def print_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    i = 1
    if message.is_multipart():
        for payload in message.get_payload():
            print(payload.get_payload(decode=True).decode("utf-8"))
            i+= 1
            print(i)
    else:
        print(message.get_payload(decode=True).decode("utf-8"))

### Eigene Funktionen

In [11]:
def markAsRead(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().modify(userId='me', id=mid, body={'removeLabelIds': ['UNREAD']}).execute()
    else:
        service.users().messages().modify(userId='me', id=message_id, body={'removeLabelIds': ['UNREAD']}).execute()
    
def deleteMail(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().delete(userId="me", id=mid).execute()
    else:
        service.users().messages().delete(userId="me", id=message_id).execute()

In [12]:
# Nötig, wenn Nachrichten von Google reinkommen.
# deleteMail(service,"17767b037c45a4d3")

### Usage of the function

In [13]:
token_path = "token.pickle"

In [14]:
service = load_service(token_path)
#messages = receive_all_messages(service)

In [15]:
#message = create_message("datathon@lumos-consulting.at", "thomas.n71@gmail.com", "Test_0", "Das ist der erste Testä")

In [16]:
#send_message(service, "me", message)

In [17]:
#messages = receive_all_messages(service) #receive_unread_messages(service)

In [18]:
def get_text_from_message(message, onlyfirstpart=True):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    retText=""
    try:
        if message.is_multipart():

            for payload in message.get_payload():
                retText+=str(payload.get_payload(decode=True).decode("utf-8"))
                if onlyfirstpart:
                    return retText
                retText+="\n"
        else:
            retText+=str(message.get_payload(decode=True).decode("utf-8"))
    except:
        message.get_payload(decode=True).decode("utf-8")
    return retText

In [19]:
import numpy as np
import torch
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer
labellist=np.arange(1,7)
PATH="C:\\Users\\Tom\\Google Drive\\data\\final_model"
recipient = "thomas.n71@gmail.com" # TODO CHANGE

def predictAbteilung(text):
    model = AutoModelForSequenceClassification.from_pretrained(PATH)
    tokenizer = AutoTokenizer.from_pretrained(PATH)
    labels = torch.tensor([1]).unsqueeze(0)
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    prediction = model(**inputs, labels=labels)
    logits=prediction.logits.detach().numpy()[0]
    probabilities=softmax(logits)
    print(probabilities)
    return arg3max(probabilities)

In [20]:
def arg3max(liste):
    retList = [(0,0),(0,0),(0,0)]
    print(liste)
    for (count,value) in enumerate(liste):
        if value > retList[0][1]:
            if value > retList[1][1]:
                if value > retList[2][1]:
                    retList[0]=retList[1]
                    retList[1]=retList[2]
                    retList[2]=(count,value)
                else:
                    retList[0]=retList[1]
                    retList[1]=(count,value)
            else:
                retList[0]=(count,value)
    return retList
        
def mail_weiterleitung(service):
    
    messages = receive_unread_messages(service)

    for mid,message in messages:
        text=get_text_from_message(message)
        print(100*"*")
        print(text)
        predictions = predictAbteilung(text)
        abteilung1 = labellist[predictions[2][0]]
        abteilung2 = labellist[predictions[1][0]]
        abteilung3 = labellist[predictions[0][0]]
        text+="""\n__________________________________________\nDiese Mail ist mit {p1:.2f} % Wahrscheinlichkeit für Abteilung {a1} bestimmt.\n\nMit {p2:.2f} % Wahrscheinlichkeit könnte Sie für Abteilung {a2} und mit {p3:.2f} % Wahrscheinlichkeit für Abteilung {a3} bestimmt sein.""".format(p1=predictions[2][1]*100,p2=predictions[1][1]*100,p3=predictions[0][1]*100,a1=str(abteilung1),a2=str(abteilung2),a3=str(abteilung3))
        if predictions[2][1] > 0.9:
            betreff="Abteilung "+str(abteilung1)+":  "+str(message["subject"])
        else:
            betreff="Unklarheit bei Zuweisung: "+str(message["subject"])
        wl_message = create_message("team4@lumos-consulting.at", recipient, betreff, text)
        send_message(service, "me", wl_message)
        markAsRead(service,mid)

In [ ]:
import time
while True:
    
    mail_weiterleitung(service)
    time.sleep(5)

1777451980dda622
Delivered-To: team4@lumos-consulting.at
Received: by 2002:a05:7000:13d3:0:0:0:0 with SMTP id v19csp2281053mal;
        Fri, 5 Feb 2021 14:30:40 -0800 (PST)
X-Received: by 2002:a62:6d06:0:b029:1d0:f7ca:59d0 with SMTP id i6-20020a626d060000b02901d0f7ca59d0mr6780878pfc.75.1612564240463;
        Fri, 05 Feb 2021 14:30:40 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1612564240; cv=none;
        d=google.com; s=arc-20160816;
        b=qeV1ijXRYaHAg6iqpzBmO9HUZx+J6ofOvvk/HkdBJxLiUkfAlhOdnVXs0YUEboyUKk
         gI5W5IgScob5GQAPfeQp4cN9G9luOKKbHIZlSK2w8qVYoyXRtk+lBt+0TLuW3o846Tnb
         f4gkxKDYxGsaYOfZWJWc7W8O9oGCz9QTNpmvG+AWvnKUI/D7sebGKl1qPXCAT00fhxaK
         BnjlsiwMwwfkj89RhbeZe+7j4NP5R93i8pIF4bCrVK9KHSmSWs6iCj01kF1+/Cbj6IF1
         /za6vidi0duUbQbWFEPQ0Yz4+yb5Mjc+emO/Jx8IaJ4sOiz6KQhnxWVwouYVM5YBtA06
         Wsyg==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=to:subject:message-id:date:from:mime-version:dkim-signat

Message Id: 1777452011bebfb0
